In [1]:
import pandas as pd
import os
import sys

sys.path+=['../']
import param
import gc

### read in matrix

In [2]:
inTranscriptFmt='/nrnb/users/btsui/Data/all_seq/rnaseq_merged/{}.transcript.{}.pickle'
transcriptMatricesDir='/nrnb/users/btsui/Data/all_seq/rnaseq_merged_chunks/'


In [7]:
inFnameS=pd.Series(os.listdir(transcriptMatricesDir))

In [8]:
#remove specie_m when running all
specie_m=inFnameS.str.contains('Mus_musculus')

In [9]:
m=inFnameS.str.contains('\.transcript\.')#&specie_m
selectedFanmeS=inFnameS[m]
inTranscriptDirs=transcriptMatricesDir+'/'+selectedFanmeS

In [10]:
selectedFanmeS.head()

0    Drosophila_melanogaster.transcript.tpm.SRR19.p...
1              Danio_rerio.transcript.tpm.SRR16.pickle
4      Mus_musculus.transcript.est_counts.SRR29.pickle
6              Danio_rerio.transcript.tpm.SRR59.pickle
7        Danio_rerio.transcript.est_counts.ERR6.pickle
dtype: object

In [11]:
transcriptNameRemoveRegex='^>|\.\d+$'

In [12]:
def geneLevelCollapse(inTranscriptDir):
    inTranscriptDf=pd.read_pickle(inTranscriptDir)
    inTranscriptDf.index=inTranscriptDf.index.str.replace(transcriptNameRemoveRegex,'')
    myFname=inTranscriptDir.split('/')[-1]
    selectedSpecies=myFname.split('.')[0]
    #print (selectedSpecies)
    ### read in annotation mapping from FASTA file
    faBaseDir='/cellar/users/btsui/Data/ensembl/release/cdna/'
    refs=pd.Series(os.listdir(faBaseDir))
    refFname=refs[refs.str.contains('^'+selectedSpecies)].iloc[0]

    refFaS=pd.read_csv(faBaseDir+refFname,sep='asfidushafosaf',header=None)[0]
    annotationS=refFaS[refFaS.str.contains('^>')]

    resolution='gene_symbol'
    myDict={'gene_symbol':6}
    myI=myDict[resolution]
    transcriptAnnotDf=annotationS.str.split('\s+',expand=True)
    transcriptToAnnotDf=transcriptAnnotDf[[0,6]]

    transcriptToAnnotDf.columns=['transcript_name',resolution]

    transcriptToAnnotDf['symbol']=transcriptToAnnotDf[resolution].str.replace('^\w+:','')
    transcriptToAnnotDf['transcript_name']=transcriptToAnnotDf['transcript_name'].str.replace(transcriptNameRemoveRegex,'')#.str.replace('\.\d+$','')
    #transcriptToAnnotDf.to_csv(rnaseqAnnotDirFmt.format(selectedSpecies))
    transcriptToSymbolS=transcriptToAnnotDf.set_index('transcript_name')['symbol']
    
    print ('mapping is unique: ',transcriptToSymbolS.index.is_unique)

    ### merge them onto symbol resolution

    print ('all transcript in symbol mapping annotation file: ',inTranscriptDf.index.isin(transcriptToAnnotDf['transcript_name']).mean())

    multI=pd.MultiIndex.from_arrays([inTranscriptDf.index.values,
                               transcriptToSymbolS.loc[inTranscriptDf.index].values])

    inTranscriptDf.index=multI
    inTranscriptDf.index.names=['transcript_name','symbol']
    ###merge at symbole level
    symbolDf=inTranscriptDf.sum(axis=0,level='symbol')
    
    outSymolDfDir=inTranscriptDir.replace('transcript',resolution)
    symbolDf.to_pickle(outSymolDfDir)
    #mmap_data_prefix=outSymolDfDir.replace('.pickle','')
    #exportDf(mmap_data_prefix,symbolDf)
    
    del (symbolDf)
    gc.collect()

    return 0

In [13]:
from multiprocessing import Pool
from tqdm import tqdm

In [14]:
with Pool(32) as p:
    statusL=list(tqdm(p.imap(geneLevelCollapse,inTranscriptDirs),total=len(inTranscriptDirs)))

  0%|          | 0/648 [00:00<?, ?it/s]/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='p

mapping is unique:  True
all transcript in symbol mapping annotation file:  1.0


/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydat

mapping is unique:  True
mapping is unique:  True
all transcript in symbol mapping annotation file:  1.0
all transcript in symbol mapping annotation file:  1.0


/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/cellar/users/btsui/anaconda3/l

...

/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:35: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


mapping is unique:  True


/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:35: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


all transcript in symbol mapping annotation file:  0.9864710666908534


/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:35: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
100%|██████████| 648/648 [21:23<00:00,  1.98s/it]


In [15]:
asdasdas

NameError: name 'asdasdas' is not defined

# scratch

### serial version

In [ ]:
#selectedSpecies='Canis_familiaris'
#countMetric='tpm'
#for selectedSpecies,countMetric in  specieMetricPairs:
transcriptNameRemoveRegex='^>|\.\d+$'
for inTranscriptDir in inTranscriptDirs:
    inTranscriptDf=pd.read_pickle(inTranscriptDir)
    inTranscriptDf.index=inTranscriptDf.index.str.replace(transcriptNameRemoveRegex,'')
    myFname=inTranscriptDir.split('/')[-1]
    selectedSpecies=myFname.split('.')[0]
    #print (selectedSpecies)
    ### read in annotation mapping from FASTA file
    faBaseDir='/cellar/users/btsui/Data/ensembl/release/cdna/'
    refs=pd.Series(os.listdir(faBaseDir))
    refFname=refs[refs.str.contains('^'+selectedSpecies)].iloc[0]

    refFaS=pd.read_csv(faBaseDir+refFname,sep='asfidushafosaf',header=None)[0]
    annotationS=refFaS[refFaS.str.contains('^>')]

    resolution='gene_symbol'
    myDict={'gene_symbol':6}
    myI=myDict[resolution]
    transcriptAnnotDf=annotationS.str.split('\s+',expand=True)
    transcriptToAnnotDf=transcriptAnnotDf[[0,6]]

    transcriptToAnnotDf.columns=['transcript_name',resolution]

    transcriptToAnnotDf['symbol']=transcriptToAnnotDf[resolution].str.replace('^\w+:','')
    transcriptToAnnotDf['transcript_name']=transcriptToAnnotDf['transcript_name'].str.replace(transcriptNameRemoveRegex,'')#.str.replace('\.\d+$','')
    #transcriptToAnnotDf.to_csv(rnaseqAnnotDirFmt.format(selectedSpecies))
    transcriptToSymbolS=transcriptToAnnotDf.set_index('transcript_name')['symbol']
    
    print ('mapping is unique: ',transcriptToSymbolS.index.is_unique)

    ### merge them onto symbol resolution

    print ('all transcript in symbol mapping annotation file: ',inTranscriptDf.index.isin(transcriptToAnnotDf['transcript_name']).mean())

    multI=pd.MultiIndex.from_arrays([inTranscriptDf.index.values,
                               transcriptToSymbolS.loc[inTranscriptDf.index].values])

    inTranscriptDf.index=multI
    inTranscriptDf.index.names=['transcript_name','symbol']
    ###merge at symbole level
    symbolDf=inTranscriptDf.sum(axis=0,level='symbol')
    
    outSymolDfDir=inTranscriptDir.replace('transcript',resolution)
    symbolDf.to_pickle(outSymolDfDir)
    #mmap_data_prefix=outSymolDfDir.replace('.pickle','')
    #exportDf(mmap_data_prefix,symbolDf)
    
    del (symbolDf)
    gc.collect()
    #symbolDf.to_pickle(outSymolDfDir)

In [ ]:
transcriptToAnnotDf['transcript_name']=transcriptToAnnotDf['transcript_name'].str.replace('^>|\.\d+$','')#.str.replace('','')


In [ ]:
#transcriptToAnnotDf..str.replace('^>|\.\d+$','')

In [ ]:
### merge into gene level, first, then do a vstack on the data. 

In [ ]:
mmap_data_prefix

In [ ]:
!ls /nrnb/users/btsui/Data/all_seq/rnaseq_merged/

In [ ]:
print (prefix)

In [ ]:
exportDf(prefix,symbolDf)

In [ ]:
#!rm  /nrnb/users/btsui/Data/all_seq/rnaseq_merged/Canis_familiaris.gene_symbol.tpm.pickle

In [ ]:
!ls /nrnb/users/btsui/Data/all_seq/rnaseq_merged/

In [ ]:
#import sys
#sys.path

In [ ]:
#transcriptToSymbolS
#import sharedVariable 

In [ ]:
#sharedVariable.exportDf

In [ ]:
#!cat /cellar/users/btsui/Project/METAMAP/code/metamap/sharedVariable.py